In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import numpy.ma as ma
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow.keras.preprocessing as prep
from sklearn.model_selection import train_test_split
from model import AugementedConvLSTM
import configparser
import argparse
import h5py
import itertools

projection_dimensions = [129,135]
end_index = 7305
# import pandas as pd
# len(pd.date_range(start='1/1/1948', end='12/31/1967'))

In [2]:
Aug_ConvLSTM_model = AugementedConvLSTM(
    projection_height = 129, 
    projection_width = 135,
    timesteps=4
    )
model = Aug_ConvLSTM_model.model(
    [32, 16, 16], 
    [9,5,3], 
    [64,32,1], 
    [9,3,5], 
    2
    )
model.load_weights('convlstm_weights_pr.h5')

2022-06-28 03:47:05.409674: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-28 03:47:08.601585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8688 MB memory:  -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5
2022-06-28 03:47:08.605508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9631 MB memory:  -> device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2022-06-28 03:47:08.608996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/r

In [3]:
def load_dataset(mask = [1,1,1,1,1], rf_idx = 1, end_index = 21185):
    filepath = r"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/05_Random_Field_India/Data"
    rf = np.array(np.load(os.path.join(filepath,rf"r{rf_idx}.npy")))[:end_index,:,:]
    X = np.empty((7,end_index,129,135))
    X[0,:,:,:] = np.array(np.load(r"MIROC-ESM.npy"))[:end_index,:,:] if mask[0] else np.array(rf)
    X[1,:,:,:] = np.array(np.load(r"elev.npy"))[:end_index,:,:] if mask[1] else np.array(rf)
    X[2,:,:,:] = np.array(np.load(os.path.join(filepath,rf"rhum.npy")))[:end_index,:,:] if mask[2] else np.array(rf)
    X[3,:,:,:] = np.array(np.load(os.path.join(filepath,rf"pres.npy")))[:end_index,:,:] if mask[3] else np.array(rf)
    X[4,:,:,:] = np.array(np.load(os.path.join(filepath,rf"uwnd.npy")))[:end_index,:,:] if mask[4] else np.array(rf)
    X[5,:,:,:] = np.array(np.load(os.path.join(filepath,rf"vwnd.npy")))[:end_index,:,:] if mask[4] else np.array(rf)
    X[6,:,:,:] = np.array(np.load(os.path.join(filepath,rf"omega.npy")))[:end_index,:,:] if mask[4] else np.array(rf)
    # print(X.shape)
    Y = np.load(rf"/home/uditbhatia/Documents/Sarth/Downscaling_AugmentedConvLSTM/05_Random_Field_India/Data/IMD.npy")[:,:end_index,:,:]
    return X, Y

def normalize(data):
    data = data - data.mean()
    data = data / data.std()
    return data

def set_data(X, Y, end_index = 21185):
    X_normalized = np.zeros((7, end_index, 129, 135))
    for i in range(7):
        X_normalized[i,] = normalize(X[i,])
    Y_normalized = normalize(Y)
    std_observed = Y.std()
    X = X_normalized.transpose(1,2,3,0)
    Y = Y_normalized.reshape(-1,129, 135, 1)
    return X, Y, std_observed

def data_generator(X,Y): # 25+5 ... 15+5 = 1962
    min_train_year = 1948
    max_train_year = 1962 #1973 #1999
    min_test_year = 1963 #1974 #2000
    max_test_year = 1967 #1979 #2005
    # 1948-1973-1974-1979
    total_years = max_test_year - min_train_year + 1
    train_years = max_train_year - min_train_year + 1
    n_days = np.max(X.shape)
    train_days = int((n_days/total_years)*train_years)
    train_x, train_y = X[:train_days], Y[:train_days]
    test_x, test_y = X[train_days:], Y[train_days:]
    time_steps = 4
    batch_size1 = 15
    train_generator = prep.sequence.TimeseriesGenerator(
        train_x, 
        train_y.reshape(-1, projection_dimensions[0], projection_dimensions[1], 1),
        length=time_steps, 
        batch_size=batch_size1
        )
    test_generator = prep.sequence.TimeseriesGenerator(
        test_x, 
        test_y.reshape(-1, projection_dimensions[0], projection_dimensions[1], 1),
        length=time_steps, 
        batch_size=batch_size1
        )
    return train_generator, test_generator

def train(clstm_model, train_generator, test_generator, load_weights = False, std_observed = 1.0):

    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

    def actual_rmse_loss(y_true, y_pred):
        return K.sqrt(K.mean(K.square((y_pred - y_true)*std_observed)))

    adam = tf.keras.optimizers.Adam(learning_rate=0.0003)
    
    clstm_model.compile(optimizer=adam, loss=root_mean_squared_error, metrics=[root_mean_squared_error, actual_rmse_loss])

    checkpoint = tf.keras.callbacks.ModelCheckpoint(f"convlstm_weights_pr_modified.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f"./Graphs/norm_csltm_pre_Graph", histogram_freq=0, write_graph=True, write_images=False)
    termnan = tf.keras.callbacks.TerminateOnNaN()
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, min_delta=0.005, min_lr=0.000004, verbose=1)
    
    callbacks_list = [checkpoint,tensorboard, reduce_lr, termnan]
    
    history = clstm_model.fit(
        train_generator, 
        callbacks=callbacks_list, 
        epochs=4, 
        validation_data=test_generator,
        verbose=1
        )
    return history

In [4]:
mask_list = [ # Pr, elev, rh, p, wnd
    [1,1,1,1,1], #0
    [1,0,1,1,1], #1
    [1,1,0,1,1], #2
    [1,1,1,0,1], #3
    [1,1,1,1,0], #4
    [0,1,1,1,0], #5
    [0,1,1,0,1], #6
    [0,1,0,1,1], #7
    [0,1,1,0,0], ##8
    [0,1,0,0,1], ##9
    [0,1,0,1,0], ##10
    [0,1,0,0,0], ###11
    [0,0,1,0,0], ###12
    [0,0,0,1,0], ###13
    [0,0,0,0,1], ###14
    [1,0,0,0,0],
    [0,0,0,0,0]
]
# np.random.seed(0)
# np.random.shuffle(mask_list)

In [5]:
for mask in itertools.islice(mask_list, 1, 5, 1):
    print(mask)
    X, Y = load_dataset(mask = mask, end_index = end_index)
    X, Y, std_observed = set_data(X,Y,end_index=end_index)
    print(std_observed)
    train_generator, test_generator = data_generator(X, Y)
    history = train(
        model,
        train_generator, 
        test_generator,
        load_weights = True,
        std_observed = std_observed
        )

[1, 0, 1, 1, 1]
5.767554285835394
Epoch 1/4


/home/uditbhatia/anaconda3/envs/Sarth-tf/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
2022-06-28 03:48:00.181232: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 7605
2022-06-28 03:48:01.114909: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2022-06-28 03:48:01.605293: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


365/365 [==============================] - ETA: 0s - loss: 0.7954 - root_mean_squared_error: 0.7953 - actual_rmse_loss: 4.5868
Epoch 00001: val_loss improved from inf to 0.75340, saving model to convlstm_weights_pr_modified.h5
365/365 [==============================] - 269s 710ms/step - loss: 0.7954 - root_mean_squared_error: 0.7953 - actual_rmse_loss: 4.5868 - val_loss: 0.7534 - val_root_mean_squared_error: 0.7524 - val_actual_rmse_loss: 4.3393 - lr: 3.0000e-04
Epoch 2/4
365/365 [==============================] - ETA: 0s - loss: 0.7681 - root_mean_squared_error: 0.7680 - actual_rmse_loss: 4.4293
Epoch 00002: val_loss improved from 0.75340 to 0.74437, saving model to convlstm_weights_pr_modified.h5
365/365 [==============================] - 262s 717ms/step - loss: 0.7681 - root_mean_squared_error: 0.7680 - actual_rmse_loss: 4.4293 - val_loss: 0.7444 - val_root_mean_squared_error: 0.7433 - val_actual_rmse_loss: 4.2873 - lr: 3.0000e-04
Epoch 3/4
365/365 [==============================] -

In [6]:
for mask in itertools.islice(mask_list, 5, 11, 1):
    print(mask)
    X, Y = load_dataset(mask = mask, end_index = end_index)
    X, Y, std_observed = set_data(X,Y,end_index=end_index)
    print(std_observed)
    train_generator, test_generator = data_generator(X, Y)
    history = train(
        model,
        train_generator, 
        test_generator,
        load_weights = True,
        std_observed = std_observed
        )

[0, 1, 1, 1, 0]
5.767554285835394
Epoch 1/4
365/365 [==============================] - ETA: 0s - loss: 0.7510 - root_mean_squared_error: 0.7509 - actual_rmse_loss: 4.3310
Epoch 00001: val_loss improved from inf to 0.71590, saving model to convlstm_weights_pr_modified.h5
365/365 [==============================] - 268s 719ms/step - loss: 0.7510 - root_mean_squared_error: 0.7509 - actual_rmse_loss: 4.3310 - val_loss: 0.7159 - val_root_mean_squared_error: 0.7150 - val_actual_rmse_loss: 4.1237 - lr: 3.0000e-04
Epoch 2/4
365/365 [==============================] - ETA: 0s - loss: 0.7454 - root_mean_squared_error: 0.7453 - actual_rmse_loss: 4.2985
Epoch 00002: val_loss improved from 0.71590 to 0.71442, saving model to convlstm_weights_pr_modified.h5
365/365 [==============================] - 263s 720ms/step - loss: 0.7454 - root_mean_squared_error: 0.7453 - actual_rmse_loss: 4.2985 - val_loss: 0.7144 - val_root_mean_squared_error: 0.7135 - val_actual_rmse_loss: 4.1150 - lr: 3.0000e-04
Epoch 3/

In [7]:
for mask in itertools.islice(mask_list, 11, 15, 1):
    print(mask)
    X, Y = load_dataset(mask = mask, end_index = end_index)
    X, Y, std_observed = set_data(X,Y,end_index=end_index)
    print(std_observed)
    train_generator, test_generator = data_generator(X, Y)
    history = train(
        model,
        train_generator, 
        test_generator,
        load_weights = True,
        std_observed = std_observed
        )

[0, 1, 0, 0, 0]
5.767554285835394
Epoch 1/4
365/365 [==============================] - ETA: 0s - loss: 0.8174 - root_mean_squared_error: 0.8173 - actual_rmse_loss: 4.7140
Epoch 00001: val_loss improved from inf to 0.76869, saving model to convlstm_weights_pr_modified.h5
365/365 [==============================] - 268s 721ms/step - loss: 0.8174 - root_mean_squared_error: 0.8173 - actual_rmse_loss: 4.7140 - val_loss: 0.7687 - val_root_mean_squared_error: 0.7677 - val_actual_rmse_loss: 4.4275 - lr: 3.0000e-04
Epoch 2/4
365/365 [==============================] - ETA: 0s - loss: 0.8155 - root_mean_squared_error: 0.8154 - actual_rmse_loss: 4.7029
Epoch 00002: val_loss did not improve from 0.76869
365/365 [==============================] - 264s 722ms/step - loss: 0.8155 - root_mean_squared_error: 0.8154 - actual_rmse_loss: 4.7029 - val_loss: 0.7719 - val_root_mean_squared_error: 0.7709 - val_actual_rmse_loss: 4.4461 - lr: 3.0000e-04
Epoch 3/4
365/365 [==============================] - ETA: 0s 